<a href="https://colab.research.google.com/github/yasyamauchi/education/blob/main/2023AIreport1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データを読み込む  

* country	国名
* child_mort	出生1000人あたりの5歳未満の子供の死亡数
* exports	人口当たり輸出額/GDP
* health	人口当たり医療費/GDP
* imports	人口当たり輸入額/GDP
* Income	人口当たり収入
* Inflation	経済成長率
* life_expec	平均寿命
* total_fer	平均出生数/女性
* gdpp	人口当たりGDP  

## 1) ファイルCountry-data.csvをToyoNet-ACEからダウンロードして，Google Colabにアップロードする  
手順は講義で説明したとおり．

##2) 次のコードを実行してデータを読み込む．最初の5行が表示される．

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

data_df = pd.read_csv('Country-data.csv')
data_df.head(5)

##3) 次のコードを実行し，全データの組み合わせの散布図を作成する

In [ ]:
sns.pairplot(data_df)

# 問題２：一次関数による近似

平均寿命を横軸，子供の死亡数を縦軸のグラフを作成する．子供の死亡数が多いほど平均寿命が低くなることを確認する．

In [ ]:
data_df_lc=data_df.loc[:,["child_mort","life_expec"]]
ax = sns.scatterplot(x='child_mort', y='life_expec', data=data_df_lc)

##【課題】一次関数$y=ax+b$で近似する  
次のコードを使用し，aの初期値は-1，bの初期値は60から試して，両方の数字の調整を繰り返して，誤差Jが最も少なくなりそうなaとb，そのときの誤差Jを答えよ．また，そのときのグラフのスクリーンショットをレポートにつけよ．

In [ ]:
# 1次関数の定義
def estimate_func(param,x):
    return (param[0]*x + param[1])

p = []
p.append(float(input('a= ')))  # 最初は-1あたりから試す
p.append(float(input('b= ')))  # 最初は60あたりから試す

x = data_df['child_mort'].to_list()
y = data_df['life_expec'].to_list()

ylin = [estimate_func(p,i) for i in x]  # y = ax + b

# 誤差を合計する
J = 0
for i in range(len(x)):
    J += (ylin[i]-y[i])**2
print('J={}'.format(J))

# グラフの描画
fig, axe = plt.subplots()
axe.set_xlabel('life_expec')  # x軸ラベル
axe.set_ylabel('child_mort')  # y軸ラベル
axe.grid()            # 罫線
#axe.set_xlim([0, 300]) # x方向の描画範囲を指定
axe.scatter(x,y)
for i in range(len(x)):
    axe.plot([x[i],x[i]],[y[i],ylin[i]],color="g",linestyle = "dashed")
axe.plot(x,ylin,color="r")
fig.tight_layout()  # レイアウトの設定
plt.show()

# 問題３：クラスタリング  

## 1) 医療費とGDPのデータだけ取り出して散布図にする  
なおGDPは1万で割る

In [ ]:
data_df_hg=data_df.loc[:,["health","gdpp"]]
data_df_hg.loc[:,"gdpp"]=data_df_hg.loc[:,"gdpp"].apply(lambda x: x/10000) # 1万で割る（※xとyの桁が乖離しているとK-Meansはうまくいかない）
ax = sns.scatterplot(x='health', y='gdpp', data=data_df_hg)
plt.rcParams["figure.figsize"] = (10, 5)
plt.xlim(0, 20)
plt.ylim(0, 10)
plt.show()

## 2)【課題】クラスター数の決定  
次のK-Means法のコードを実行して得られた図から，クラスターの適切な数を求めよ

横軸はクラスター数，縦軸は損失(各クラスタの中心からデータへの二乗誤差の総和とのこと)となる

In [ ]:
X = data_df_hg.values
from sklearn.cluster import KMeans
objective_vals = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k, init='k-means++', n_init=10, max_iter=300, tol=0.0001, random_state=0)
    km = km.fit(X)
    objective_vals.append(km.inertia_)
plt.plot(K, objective_vals, 'bx-')
plt.xlabel('k')
plt.ylabel('Objective values')
plt.title('Elbow Method K-Means')
plt.show()

## 3)クラスター当たりの国数を表示する  
このコードの最初の行の"n_clusters=10"の**"10"の代わりに，先ほど決めたクラスターの数を記入**して実行すること

In [ ]:
pred = KMeans(n_clusters=10, init='k-means++', n_init=10, max_iter=300, tol=0.0001, random_state=0).fit_predict(X)
data_df_hgc=data_df_hg
data_df_hgc['cluster']=pred
data_df_hgc['cluster'].value_counts()

## 4)クラスター当たりの医療費支出とGDPの平均を求める

In [ ]:
data_df_hgc.groupby('cluster').mean()

##5)【課題】クラスターの可視化  
次のコードを実行し，クラスターを図示し，画面キャプチャでレポートに添付すること．

In [ ]:
ax = sns.scatterplot(x='health', y='gdpp', hue='cluster',data=data_df_hgc, palette='colorblind')
plt.rcParams["figure.figsize"] = (10, 5)
plt.xlim(0, 20)
plt.ylim(0, 10)
plt.show()

以上